In [2]:
from guidance import models, gen
Lucid = models.Transformers('TheBloke/dolphin-2.6-mistral-7B-dpo-GPTQ', device_map="cuda")

In [1]:
import subprocess

def run_command(command):
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        output = result.stdout.decode('utf-8')
        error = result.stderr.decode('utf-8')
        return output, error
    except subprocess.CalledProcessError as e:
        return None, e.stderr.decode('utf-8')

# Example: Run 'dir' command in the Windows terminal
command = 'dir'
output, error = run_command(command)

if output:
    print("Command output:")
    print(output)
else:
    print("Command failed with error:")
    print(error)


Command output:
 Volume in drive C has no label.
 Volume Serial Number is 0CF8-41F0

 Directory of c:\Users\User\Desktop\Projects\Lucid

01/10/2024  03:06 PM    <DIR>          .
01/07/2024  07:54 PM    <DIR>          ..
07/20/2023  08:07 AM                66 .gitattributes
08/29/2023  04:45 PM                19 .gitignore
11/01/2023  03:34 PM             5,463 1.1.1
10/18/2023  03:09 PM               124 Ask.md
07/28/2023  11:48 PM    <DIR>          cool assets
12/17/2023  03:16 PM    <DIR>          Dataset
12/02/2023  09:39 PM    <DIR>          dictionary
08/19/2023  09:46 PM    <DIR>          dpet
01/10/2024  03:39 PM            17,334 guidance_playground.ipynb
07/22/2023  02:12 PM    <DIR>          History
01/03/2024  03:29 PM    <DIR>          MarkDowns
01/07/2024  10:34 PM    <DIR>          OLD
07/22/2023  03:53 PM    <DIR>          test
01/03/2024  07:40 PM    <DIR>          __pycache__
               5 File(s)         23,006 bytes
              11 Dir(s)  214,586,195,968 bytes f

In [3]:
from guidance import select, gen, models, regex
import guidance
import subprocess

# Additional tool for Windows Command Line
@guidance(stateless=False)
def windows_command_line(lm, command):
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        output = result.stdout.decode('utf-8')
        error = result.stderr.decode('utf-8')
        lm += f'\nObservation {regex(r"[0-9]+")}: {output}\n'
        if error:
            lm += f'\nObservation {regex(r"[0-9]+")}: Error - {error}\n'
    except subprocess.CalledProcessError as e:
        lm += f'\nObservation {regex(r"[0-9]+")}: Error - {e.stderr.decode("utf-8")}\n'
    return lm

tools = {
    "windows_command_line": "Execute commands in the Windows command line."
}
tool_map = {
    "windows_command_line": windows_command_line
}
task = "Create a folder on the Desktop called \"Lucid_Folder\""
prompt = """Do the following task as best you can. You have access only to the following tools:

{tools}

Use the following format:

Task: the task you must complete
Thought 1: you should always think about what to do
Action 1: the action to take, has to be one of {tool_names}
Observation 1: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought N: The original task is complete.
Final Answer: The original task is complete.
Done.

Example:
Task: Find the square root of the age of Brad Pitt?
Thought 1: I should find out how old Brad Pitt is.
Action 1: age(Brad Pitt)
Observation 1: 56
Thought 2: I should find the square root of 56.
Action 2: sqrt(56)
Observation 2: 7.48
Thought 3: I now know the final answer.
Final Answer: 7.48
Done.

Task: {query}
"""
print(type(tools))
print(tools.keys())
@guidance
def react_agent(lm, task, input_tools, max_rounds=50):
    print(input_tools)
    tool_names = list(input_tools.keys())
    lm += prompt.format(tools=input_tools, tool_names=tool_names, query=task)
    i = 1
    while True:
        lm += f'Thought {i}: ' + gen(name='thought', suffix='\n',temperature=0.8)
        print(lm['thought'])
        if 'original task is complete' in lm['thought'] or i == max_rounds:
            lm += 'Final Answer: The original task is complete.\nDone.\n'
            break
        lm += f'Action {i}: ' + select(tool_names, name='act') + '\n'
        lm += '(' + gen(name='arg', suffix=')',temperature=0.8) + '\n'
        if lm['act'] in tool_map:
            lm += tool_map[lm['act']](lm['arg'])
        i += 1
    return lm

# Example usage
tools = {
    "windows_command_line": "Execute commands in the Windows Terminal."
}

# Create a new model instance
#Lucid = models.LanguageModel()

# Call the react_agent function with the new model instance
lm = Lucid
lm += react_agent(task, tools)


In [ ]:
prompt = """Do the following task as best you can. You have access only to the following tools:

{tools}

Use the following format:

Task: the task you must complete
Thought 1: you should always think about what to do
Action 1: the action to take, has to be one of {tool_names}
Observation 1: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought N: The original task is complete.
Final Answer:The original task is complete.
Done.

Example:
Task: Find the square root of the age of Brad Pitt?
Thought 1: I should find out how old Brad Pitt is.
Action 1: age(Brad Pitt)
Observation 1: 56
Thought 2: I should find the square root of 56.
Action 2: sqrt(56)
Observation 2: 7.48
Thought 3: I now know the final answer.
Final Answer: 7.48
Done.

Question: {query}
"""